In [ ]:
import pandas as pd
import numpy as np

import MySQLdb
from pandas.io import sql

conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor() 

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()

In [ ]:
# creating concepts for chartevents

In [ ]:
df_chartevents_mimic = pd.read_sql('SELECT distinct * FROM mimic4.d_items where linksto = "chartevents" and param_type!="Checkbox" and (category="Routine Vital Signs" or category="General")', con = mimicconn)
df_chartevents_mimic

In [ ]:
df_chartevents_mimic['label'] = df_chartevents_mimic['label'].str.lower()
df_chartevents_mimic['abbreviation'] = df_chartevents_mimic['abbreviation'].str.lower()

In [ ]:
df_chartevents_mimic = df_chartevents_mimic.drop_duplicates(subset = 'label')

In [ ]:
df_chartevents_mimic.nunique()

In [ ]:
df_chartevents_numeric_concept = df_chartevents_mimic[df_chartevents_mimic['param_type']=='Numeric']
df_chartevents_numeric_concept = df_chartevents_numeric_concept[['label', 'abbreviation']]

In [ ]:
df_chartevents_numeric_concept['class_id'] = 1
df_chartevents_numeric_concept['datatype_id'] = 1
df_chartevents_numeric_concept['date_created']= '2022-01-01'
df_chartevents_numeric_concept['creator'] = 1

import uuid
df_chartevents_numeric_concept['uuid'] = [uuid.uuid4() for x in range(len(df_chartevents_numeric_concept))]

In [ ]:
df_chartevents_numeric_concept = df_chartevents_numeric_concept.rename(columns = {'label':'description', 'abbreviation':'short_name'})
df_chartevents_numeric_concept

In [ ]:
column_names = ['short_name', 'description', 'datatype_id', 'class_id', 'creator', 'date_created', 'uuid']
df_chartevents_numeric_concept = df_chartevents_numeric_concept.reindex(columns = column_names)
len(df_chartevents_numeric_concept.index)

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_chartevents_numeric_concept.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_chartevents_numeric_concept.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

In [ ]:
df_chartevents_text_concept = df_chartevents_mimic[df_chartevents_mimic['param_type']=='Text']
df_chartevents_text_concept = df_chartevents_text_concept[['label', 'abbreviation']]
df_chartevents_text_concept

In [ ]:
df_chartevents_text_concept['class_id'] = 1
df_chartevents_text_concept['datatype_id'] = 3
df_chartevents_text_concept['date_created']= '2022-01-01'
df_chartevents_text_concept['creator'] = 1

import uuid
df_chartevents_text_concept['uuid'] = [uuid.uuid4() for x in range(len(df_chartevents_text_concept))]

In [ ]:
df_chartevents_text_concept = df_chartevents_text_concept.rename(columns = {'label':'description', 'abbreviation':'short_name'})
df_chartevents_text_concept

In [ ]:
df_chartevents_list = list(df_chartevents_text_concept['description'])
df_chartevents_list

In [ ]:
column_names = ['short_name', 'description', 'datatype_id', 'class_id', 'creator', 'date_created', 'uuid']
df_chartevents_text_concept = df_chartevents_text_concept.reindex(columns = column_names)

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_chartevents_text_concept.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_chartevents_text_concept.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

# concept numeric for numeric value chartevents

In [ ]:
df_chartevents_mimic

In [ ]:
df_chartevents_mimic = df_chartevents_mimic[df_chartevents_mimic['param_type']=='Numeric']
len(df_chartevents_mimic.index)

In [ ]:
df_chartevents = pd.read_sql("SELECT distinct itemid, valueuom from mimic4.chartevents where itemid IN (SELECT distinct itemid FROM mimic4.d_items where linksto = 'chartevents' and param_type!='Checkbox' and (category='Routine Vital Signs' or category='General') and hadm_id IS NOT NULL AND value IS NOT NULL AND value NOT regexp '[[:alpha:]]')", con = mimicconn)

In [ ]:
df_chartevents_1 = df_chartevents.sort_values(by = 'valueuom', na_position='last', ascending=False)
df_chartevents_1 = df_chartevents_1.drop_duplicates(subset='itemid', keep='first')
len(df_chartevents_1.index) 

In [ ]:
df_chartevents_final = pd.merge(df_chartevents_mimic, df_chartevents_1, on="itemid")
len(df_chartevents_final.index) 

In [ ]:
df_concept = pd.read_sql('select  * from `concept`', con = conn)
df_concept

In [ ]:
chartevents_concept = df_concept[107149 : 107213]
chartevents_concept

In [ ]:
chartevents_concept_dict = dict(zip(chartevents_concept.description, chartevents_concept.concept_id))
len(chartevents_concept_dict)

In [ ]:
df_chartevents_final['concept_id'] = df_chartevents_final['label'].map(chartevents_concept_dict) 
df_chartevents_final

In [ ]:
df_chartevents_final = df_chartevents_final[['valueuom', 'lownormalvalue', 'highnormalvalue', 'concept_id']]
df_chartevents_final = df_chartevents_final.rename(columns={"valueuom": "units", "lownormalvalue":"low_normal", "highnormalvalue": "hi_normal"})
df_chartevents_final

In [ ]:
df_chartevents_final= df_chartevents_final.replace({np.NAN: None})
df_chartevents_final

In [ ]:
column_names = ['concept_id', 'hi_normal', 'low_normal', 'units']
df_chartevents_final = df_chartevents_final.reindex(columns = column_names )
df_chartevents_final

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_chartevents_final.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_chartevents_final.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept_numeric` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

In [ ]:
openmrscursor.execute('UPDATE concept_numeric SET allow_decimal = 1')
conn.commit()

In [ ]:
df_concept_numeric = pd.read_sql('select  * from `concept_numeric`', con = conn)
df_concept_numeric